In [1]:
[a for a in [1,2,3] if a not in {'OX'}]

[1, 2, 3]

In [1]:
import torch as th
import itertools
import atomium
import pandas as pd
import params

In [5]:

data = atomium.fetch('2lyz')
data = data.model

In [4]:
aa_atoms = ['N','C','O','CA','CB','CG','CD','CE','CZ','OD','NH','NE','OG','ND','SG','OE','CH','NZ','OH','SD','OX']

atom_id = {ch : i for i, ch in enumerate(charge.keys())}
AT_INT = {k : i for i, k in enumerate(aa_atoms)}
INT_AT = {v:k for k, v in AT_INT.items()}

at_at_list = [[f'{at1}-{at2}' for at1 in aa_atoms] for at2 in aa_atoms]
AT_AT_name = list(itertools.chain(*at_at_list))
ATAT_INT = {k : i for i,k in enumerate(AT_AT_name)}
num_atat_feats = len(ATAT_INT)


In [5]:
HYDROPHOBIC = {'ALA', 'VAL', 'LEU', 'ILE', 'MET', 'PHE', 'TRP', 'RPO', 'TYR'}
AROMATIC = {'TRP', 'TYR', 'PHE'}
CATION_PI = {'CG', 'CD', 'CE', 'CZ'}
SALT_BRIDGE_C1 = {'NH', 'NE'}
SALT_BRIDGE_C2 = {'OE', 'OE'}
HYDROGEN_ACCEPTOR = {'NH', 'OH'}
HYDROGEN_DONOR = {'F', 'N', 'O'}

In [115]:
nan_type = float('nan')
atoms = []
name = []
ca_xyz = []
cb_xyz = []
residues = []
residues_name = []
is_side_chain = []
res_at_num = []
i = 0
for chain in data.chains():
    for res in chain.residues():
        r_at_name = [r.name for r in res.atoms()]
        res_at_num.append(len(r_at_name))
        for atom in res.atoms():
            n = atom.name
            if n == 'CA':
                ca_xyz.append(atom.location)
            elif n == 'CB':
                cb_xyz.append(atom.location)
            elif len(n) == 3:
                n = n[:2]
            name.append(n)
            is_side_chain.append(atom.is_side_chain)
            atoms.append(atom.location)
            residues.append(i)
            residues_name.append(res.name)
        if 'CB' not in r_at_name:
            cb_xyz.append((nan_type, nan_type, nan_type))
        res_at_num.append(j)
        i += 1

In [72]:
is_res_hf = [True if r in HYDROPHOBIC else False for r in residues_name]
#is_at_rg = [True if at is False else False for at in is_side_chain ]
is_at_hb_a = [True if r in HYDROGEN_ACCEPTOR else False for r in name]
is_at_hb_d = [True if r in HYDROGEN_DONOR else False for r in name]

is_res_ar = [True if r in AROMATIC else False for r in residues_name]

is_res_cpi = [True if at in CATION_PI else False for at in name]

is_res_arg = [True if r in {'ARG'} else False for r in residues_name]

is_at_sb_c1 = [True if at in SALT_BRIDGE_C1 else False for at in name]
is_res_sb_c1 = [True if at in {'ARG', 'LYS'} else False for at in residues_name]
is_at_sb_c2 = [True if at in {'ARG', 'GLU'} else False for at in name]
is_res_sb_c2 = [True if at in SALT_BRIDGE_C2 else False for at in residues_name]

In [116]:
name_base = [n[0] for n in name]
at_charge = [charge[n] for n in name_base]
at_vdw = []
atom_arr = [atom_id[n] for n in name_base]

at_xyz = th.FloatTensor(atoms)
at_dist = th.cdist(at_xyz, at_xyz)
at_id = th.LongTensor(atom_arr)
at_is_side = th.BoolTensor(is_side_chain)
at_is_hba = th.BoolTensor(is_at_hb_a)
at_is_hbd = th.BoolTensor(is_at_hb_d)

res_id = th.LongTensor(residues)
res_xyz = th.FloatTensor(ca_xyz)
res_dist = th.cdist(res_xyz, res_xyz)
res_cb = th.FloatTensor(cb_xyz)

In [74]:
atat_charge = th.FloatTensor(at_charge).view(-1, 1)
atat_charge = atat_charge * th.FloatTensor(at_charge).view(1, -1)

at_dist_inv = 1e-6/th.pow(at_dist + 1e-2, 2)

In [7]:
disulfde = (at_id == 4) & (at_dist < 2.2)
hydrophobic = (at_dist < 5.0) & (at_is_side == False) & th.BoolTensor(is_res_hf)
cation_pi = (at_dist < 6) & th.BoolTensor(is_res_cpi)
arg_arg = (at_dist < 5.0) & th.BoolTensor(is_res_arg)
hbond = at_is_hba,view(-1, 1) & at_is_hba.view(1, -1)

sb_tmp1 = th.BoolTensor(is_at_sb_c1).view(-1, 1) & th.BoolTensor(is_at_sb_c2).view(1, -1)
sb_tmp2 = th.BoolTensor(is_res_sb_c1).view(-1, 1) & th.BoolTensor(is_res_sb_c2).view(1, -1)

salt_bridge = sb_tmp1 & (at_dist < 5.0) & sb_tmp2

feats = th.cat((disulfde.unsqueeze(2),
               hydrophobic.unsqueeze(2),
               cation_pi.unsqueeze(2),
               arg_arg.unsqueeze(2),
               salt_bridge.unsqueeze(2),
               hbond.unsqueeze(2)), dim=2)
feats = feats.float()
coulomb_force = at_dist_inv * atat_charge
feats = th.cat((feats, 
                coulomb_force.unsqueeze(2)), dim=2)

NameError: name 'at_id' is not defined

In [83]:
efeat_list = list()
first_dim_split = feats.split(res_at_num, 0)
for i in range(len(res_at_num)):
    efeat_list.extend(list(first_dim_split[i].split(res_at_num, 1)))

In [84]:
u, v = th.where(res_dist < 8)
uv = th.where(res_dist.ravel() < 8)[0]

In [160]:
feats_at = th.cat([efeat_list[e].sum((0,1), keepdim=True) for e in uv], dim=0)

In [161]:
feats_at.shape

torch.Size([1387, 1, 6])

In [16]:
efeats = th.zeros_like(res_dist)


In [165]:
cb1 = th.linalg.norm(res_cb - res_xyz, dim=1, keepdim=True)
cb2 = cb1.clone().swapdims(0, 1)
tn_cb12 = cb1 / (cb2 + 1e-2)
tn_cb12[th.isnan(tn_cb12)] = -1

inv_ca12 = 1/(res_dist - 1e-3)

res_id_short = th.arange(0, res_id.max()+1, 1)
is_seq = th.abs(res_id_short.unsqueeze(0) - res_id_short.unsqueeze(1))
is_self = is_seq == 0
is_seq_0 = is_seq == 1
is_seq_1 = is_seq == 2
is_struct_0 = ~is_seq_0


feats_res = th.cat((tn_cb12.unsqueeze(2),
                   inv_ca12.unsqueeze(2),
                    is_self.unsqueeze(2),
                   is_seq_0.unsqueeze(2),
                   is_seq_1.unsqueeze(2),
                   is_struct_0.unsqueeze(2)), dim=2)
feats_res = feats_res[u,v]

In [167]:
feats_all = th.cat((feats_at.squeeze(), feats_res), dim=-1)

In [170]:
feats_all.mean(0)

tensor([ 0.0000e+00,  1.0887e-01,  2.7325e-01,  3.6770e-02,  0.0000e+00,
         3.0889e-03,  7.1780e-01, -7.7832e+01,  9.3006e-02,  1.8457e-01,
         1.8313e-01,  8.1543e-01])

In [ ]:
dist_where = (dist < 2)
res1, res2 = th.nonzero(dist_where, as_tuple=True)

In [ ]:
ca_adj = res_dist < 8

In [ ]:
th.index_copy_()

In [ ]:
aa1, aa2 = res_num[res1], res_num[res2]
at1, at2 = atom_id[res1], atom_id[res2]
inter = aa1 != aa2
inter1, inter2 = aa1[inter], aa2[inter]

In [ ]:
print('num:', inter.shape)
bond_list = list()
bond_dict = {i : [] for i in range(num_resid)}
for i in range(inter1.shape[0]-1):
    rid1, rid2 = inter1[i].item(), inter2[i].item()
    atid1, atid2 = at1[i].item(), at2[i].item()
    #print(rid1, ' - ', rid2, ': ', INT_AT[atid1], '-',  INT_AT[atid2])
    atat_name = INT_AT[atid1]+'-'+INT_AT[atid2]
    bond_list.append(atat_name)
    bond_dict[rid1].append(atat_name)

In [ ]:


def bond_to_vector(bond_dict):
    stack = list()
    for res, bond_list in bond_dict.items():
        
        bonds_unique = set(bond_list)
        bonds = th.zeros(params.num_atat_feats)
        
        if bonds_unique:
            
            bonds_id = [params.ATAT_INT[b] for b in bonds_unique if b in params.ATAT_INT]
            bonds[bonds_id] = 1
        stack.append(bonds.unsqueeze(0))
    feats = th.cat(stack, 0)
    return feats